# Съдържание
- [За Guidance](../../../../code/01.Introduce)
- [Настройка](../../../../code/01.Introduce)
- [Непрекъснато генериране](../../../../code/01.Introduce)
- [Говорене за Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Избор](../../../../code/01.Introduce)
- [Верига на мисълта](../../../../code/01.Introduce)
- [Генериране на JSON](../../../../code/01.Introduce)
- [Генериране на HTML](../../../../code/01.Introduce)


# За Guidance
Guidance е доказана библиотека с отворен код за Python, предназначена за контролиране на изходите на всякакви езикови модели (LM). С едно API извикване можете да изразите (в Python) точните програмни ограничения, които моделът трябва да спазва, и да генерирате структуриран изход в JSON, Python, HTML, SQL или друга структура, която изисква конкретният случай на употреба.

Guidance се различава от традиционните техники за създаване на подсказки. Тя налага ограничения, като насочва модела токен по токен в слоя за извеждане, което води до по-качествени резултати и намалява разходите и латентността с до 30–50% при използване за силно структурирани сценарии.

За да научите повече за Guidance, посетете [публичното хранилище в GitHub](https://github.com/guidance-ai/guidance) или гледайте [сесията Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) на Microsoft Build.


# Настройка
1. Инсталирайте Guidance с `pip install guidance --pre`
2. Разположете мини крайна точка Phi 3.5 в Azure, като посетите https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml и натиснете бутона "Deploy"
3. Съхранете API ключа на вашата крайна точка в променлива на средата, наречена `AZURE_PHI3_KEY`, и URL адреса в променлива на средата, наречена `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Неконтролирано генериране
Текст може да бъде генериран без никакви ограничения, използвайки функцията `gen()`. Това е същото като използването на модела без Guidance.

## Форматиране на чат
Както много модели за чат, Phi-3 очаква съобщенията между потребителя и асистента да бъдат в специфичен формат. Guidance поддържа шаблона за чат на Phi-3 и ще управлява форматирането на чата вместо вас. За да създадете редове на разговор, поставете всяка част от разговора в блок `with user()` или `with assistant()`. Блокът `with system()` може да се използва за задаване на системното съобщение.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Спестяване на токени
В силно структурирани сценарии Guidance може да пропуска токени и да генерира само необходимите токени, подобрявайки производителността, увеличавайки ефективността и спестявайки разходи за API. Генерираните токени са показани в този ноутбук с подчертана фонова област. Принудителните токени са показани без подчертаване и струват същото като входните токени, които се оценяват на една трета от цената на изходните токени.

*Забележка:* Първият пример с неконтролирано генериране не успя да принуди никакви токени, защото не предоставихме ограничения.


# Говорейки за Phi 3  
С Guidance можете лесно да вмъкнете текст в отговорите на модела. Това може да бъде полезно, ако искате да насочите изхода на модела в определена посока.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Ограничаване с regex
В предишния пример Phi 3 отговори с допълнителни обяснения след като даде отговор на въпроса с `Canberra`. За да ограничите изхода на модела до точно една дума, може да се използва regex.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

С помощта на регулярния израз се генерира само думата `Canberra`.


# Избор от множество опции
Когато са известни някои възможни опции, можете да използвате функцията `select()`, за да накарате модела да избере от списък с варианти.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

С `select()` беше генериран само токенът `Can`. Тъй като `Canberra` е единствената опция, която може да завърши отговора, останалите токени бяха принудени.


# Верига на мисълта
Верига на мисълта е техника, която може да помогне за подобряване на качеството на изхода на модела, като го насърчи да обработва проблема стъпка по стъпка. Обикновено, за да се достигне до окончателен отговор, са необходими няколко завъртания на подканата. Първо, инструктирайте модела да мисли стъпка по стъпка. След това, подканете модела отново да предостави окончателния отговор. При стандартните API за чат инференция това изисква 2 API извиквания, а генерираната от модела „верига на мисълта“ се таксува два пъти – веднъж като изходни токени, когато моделът я генерира, и след това отново като входни токени за второто извикване. С Guidance, целият многостъпков процес се обработва и таксува като част от едно API извикване, което намалява разходите и времето за реакция.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Генериране на JSON
Guidance може да се използва за гарантиране на генериране на JSON, който съответства на JSON схема или модел на pydantic, като например схемата за потребителски профил, показана тук.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Генериране на HTML

Guidance може да се използва за генериране на код и за спазване на синтактичните изисквания на програмния език. В този раздел ще създадем малка Guidance програма за писане на много опростени HTML уебстраници.

Ще разделим уебстраницата на по-малки секции, всяка със своя собствена Guidance функция. След това тези секции ще бъдат комбинирани в нашата финална функция за създаване на HTML уебстраница. 
След това ще изпълним тази функция с Guidance-активиран модел в Azure AI.

*Забележка:* Това няма да бъде напълно функционален HTML генератор; целта е да се покаже как можете да създавате структурирани изходни данни според вашите индивидуални нужди.

Започваме с импортиране на необходимото от Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML уеб страниците са силно структурирани, и ние ще „принудим“ тези части на страницата, използвайки Guidance.  
Когато изрично изискваме текст от модела, трябва да се уверим, че той не включва нищо, което би могло да бъде таг – тоест, трябва да изключим символите '<' и '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

След това можем да използваме тази функция, за да генерираме текст в произволен HTML таг:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Сега нека създадем заглавие на страницата. Като част от това, трябва да генерираме заглавие на страницата:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Тялото на HTML страницата ще бъде запълнено със заглавия и параграфи.  
Можем да дефинираме функция за всяко от тях:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Сега, функцията за определяне на тялото на самия HTML.  
Това използва `select()` с `recurse=True`, за да генерира множество заглавия и параграфи:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

След това стигаме до функцията, която генерира цялостната HTML страница.  
Добавяме началния HTML таг, след това генерираме заглавната част, след това тялото, и накрая добавяме крайния HTML таг:  


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Предоставяме удобен за потребителя обвивка, която ще ни позволи:
- Да задаваме температурата на генериране
- Да заснемаме генерираната страница от обекта Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

След това можем да запишем изхода в файл:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

И [виж резултата](../../../../code/01.Introduce/sample_page.html).



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
